In [2]:
import numpy as np
import numpy.ma as ma
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from habitablePlanets import habitable_planets

exoplanets = pd.read_csv("exoplanets3.csv", sep=",")
training_data = np.array(exoplanets[:])

'''print(training_data.shape)
print(training_data)'''

habitable_x = np.array([np.array(training_data[0, :])])
habitable_y = np.array([0])
uninhabitable_x = np.array([np.array(training_data[0, :])])
uninhabitable_y = np.array([0])

for i in training_data:
    if i[0] in habitable_planets:
        habitable_x = np.append(habitable_x, np.array([np.array(i)]), axis = 0)
        habitable_y = np.append(habitable_y, 1)
        

for i in training_data:
    if i[0] not in habitable_planets:
        uninhabitable_x = np.append(uninhabitable_x, np.array([np.array(i)]), axis = 0)
        uninhabitable_y = np.append(uninhabitable_y, 0)

        
training_x = np.append(habitable_x[:, 1:], uninhabitable_x[:, 1:], axis = 0)
training_y = np.append(habitable_y, uninhabitable_y, axis = 0)

habitable_x = habitable_x[:, 1:]
uninhabitable_x = uninhabitable_x[:, 1:]

print(habitable_x.shape, uninhabitable_x.shape, training_x.shape)
print(habitable_y.shape, uninhabitable_y.shape, training_y.shape)

(60, 25) (4366, 25) (4426, 25)
(60,) (4366,) (4426,)


In [3]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [4]:
training_x = np.asarray(training_x).astype(np.float32)
training_x = np.where(np.isnan(training_x), ma.array(training_x, mask=np.isnan(training_x)).mean(axis=0), training_x)
habitable_x = np.asarray(habitable_x).astype(np.float32)
habitable_x = np.where(np.isnan(habitable_x), ma.array(habitable_x, mask=np.isnan(habitable_x)).mean(axis=0), habitable_x)
uninhabitable_x = np.asarray(uninhabitable_x).astype(np.float32)
uninhabitable_x = np.where(np.isnan(uninhabitable_x), ma.array(uninhabitable_x, mask=np.isnan(uninhabitable_x)).mean(axis=0), uninhabitable_x)

In [5]:
oversample = SMOTE()
x_res, y_res = oversample.fit_resample(training_x, training_y)
print(x_res.shape, y_res.shape)

(8734, 25) (8734,)


In [6]:
from collections import Counter

In [7]:
print("OG dataset shape: {}".format(Counter(training_y)))
print("New dataset shape: {}".format(Counter(y_res)))

OG dataset shape: Counter({0: 4367, 1: 59})
New dataset shape: Counter({0: 4367, 1: 4367})


In [8]:
from imblearn.over_sampling import RandomOverSampler
os = SMOTE(sampling_strategy = 1)
x_train_res, y_train_res = os.fit_resample(training_x, training_y)
print(x_train_res.shape, y_train_res.shape)

(8734, 25) (8734,)


In [25]:
%%time
model = Sequential()
model.add(Dense(18, input_dim = 25, activation="sigmoid"))
model.add(Dense(18, activation="relu"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss = "binary_crossentropy", optimizer="adam", metrics=['accuracy'])

model.fit(x_train_res, y_train_res, epochs = 100, batch_size=10)

_, accuracy = model.evaluate(x_train_res, y_train_res)
print("Accuracy: %.2f" % (accuracy * 100))

Epoch 1/100
874/874 [==============================] - 1s 1ms/step - loss: 0.5896 - accuracy: 0.6859
Epoch 2/100
874/874 [==============================] - 1s 2ms/step - loss: 0.4727 - accuracy: 0.7875
Epoch 3/100
874/874 [==============================] - 1s 1ms/step - loss: 0.4658 - accuracy: 0.8028
Epoch 4/100
874/874 [==============================] - 1s 1ms/step - loss: 0.4088 - accuracy: 0.8178
Epoch 5/100
874/874 [==============================] - 1s 2ms/step - loss: 0.4139 - accuracy: 0.8020: 0s - loss: 0.4185 - accuracy: 0. - ETA: 0s -
Epoch 6/100
874/874 [==============================] - 1s 2ms/step - loss: 0.3901 - accuracy: 0.8287
Epoch 7/100
874/874 [==============================] - 1s 2ms/step - loss: 0.3818 - accuracy: 0.8295
Epoch 8/100
874/874 [==============================] - 1s 1ms/step - loss: 0.3866 - accuracy: 0.8232
Epoch 9/100
874/874 [==============================] - 1s 2ms/step - loss: 0.3892 - accuracy: 0.8163
Epoch 10/100
874/874 [=======================

Epoch 79/100
874/874 [==============================] - 1s 1ms/step - loss: 0.1317 - accuracy: 0.9646
Epoch 80/100
874/874 [==============================] - 1s 1ms/step - loss: 0.1353 - accuracy: 0.9636: 0s - loss: 0.144
Epoch 81/100
874/874 [==============================] - 1s 1ms/step - loss: 0.1350 - accuracy: 0.9624
Epoch 82/100
874/874 [==============================] - 1s 1ms/step - loss: 0.1388 - accuracy: 0.9627
Epoch 83/100
874/874 [==============================] - 1s 1ms/step - loss: 0.1435 - accuracy: 0.9591
Epoch 84/100
874/874 [==============================] - 1s 2ms/step - loss: 0.1324 - accuracy: 0.9642
Epoch 85/100
874/874 [==============================] - 1s 2ms/step - loss: 0.1430 - accuracy: 0.9596: 0s - loss: 0.1499 - accu
Epoch 86/100
874/874 [==============================] - 1s 1ms/step - loss: 0.1338 - accuracy: 0.9588
Epoch 87/100
874/874 [==============================] - 1s 1ms/step - loss: 0.1230 - accuracy: 0.9623
Epoch 88/100
874/874 [================

In [26]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size = 0.2, random_state = 0)



In [24]:
%%time
print(model.evaluate(x_test, y_test))

ValueError: in user code:

    C:\Users\Aland Liu\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1224 test_function  *
        return step_function(self, iterator)
    C:\Users\Aland Liu\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1215 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Aland Liu\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Aland Liu\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Aland Liu\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Aland Liu\anaconda3\lib\site-packages\tensorflow\python\keras\engine\train

In [102]:
test_x_2 = np.array([np.array(training_data[0, 1:])])
for i in training_data:
    if i[0] == "Teegarden's Star b":
        test_x_2 = np.append(test_x_2, np.array([i[1:]]), axis = 0)
        break

test_x_2 = np.asarray(test_x_2).astype(np.float32)
test_x_2 = np.where(np.isnan(test_x_2), ma.array(test_x_2, mask=np.isnan(test_x_2)).mean(axis=0), test_x_2)

test_y_2 = np.array([0, 1])

print(model.evaluate(test_x_2, test_y_2))

1/1 [==============================] - 0s 1ms/step - loss: 0.0089 - accuracy: 1.0000
[0.008855640888214111, 1.0]
